In [1]:
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
market = Market()
strategy = Strategy()

In [3]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()
strategy.connect()
sim = strategy.retrieve("non_financial_sim")
strategy.disconnect()

In [4]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [5]:
final_sim = prices.merge(sim,on=["year","quarter","week","ticker"],how="left").dropna().reset_index()
final_sim = final_sim[["year","quarter","date","ticker","adjClose","weekly_price_regression_prediction","score"]]

In [6]:
final_sim["delta"] = (final_sim["weekly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]

In [8]:
start = final_sim["date"].min()
end = final_sim["date"].max()
date = start
seats = 3
trades = []
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in tqdm(range(70,100,10)):
        date = start
        while date <= end:
            req = float(score/100)
            if date.weekday() > 4:
                date = date + timedelta(days=1)
            try:
                todays_sim = final_sim[(final_sim["date"]==date) & (final_sim["delta"] > 0) \
                                      & (final_sim["score"] >= req)].sort_values("delta",ascending=False)
                if todays_sim.index.size < 1:
                    date = date + timedelta(days=1)
                else:
                    for seat in range(min(seats,todays_sim.index.size)):
                        try:
                            trade = todays_sim.iloc[seat]
                            ticker = trade["ticker"]
                            buy_price = trade["adjClose"]
                            exit_price = buy_price * (1+delta)
                            max_date = date + timedelta(days=7)
                            exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                (final_sim["date"] > date) & \
                                              (final_sim["date"] <= max_date) & \
                                              (final_sim["adjClose"] >= exit_price)
                                            ]
                            if exits.index.size < 1:
                                exits = final_sim[(final_sim["ticker"]==ticker) & 
                                              (final_sim["date"] > max_date)].sort_values("date",ascending=True)
                                sell_trade = exits.iloc[0]
                            else:
                                sell_trade = exits.iloc[0]
                            trade["sell_date"] = sell_trade["date"]
                            trade["sell_price"] = sell_trade["adjClose"]
                            trade["delta"] = (trade["sell_price"] - trade["adjClose"]) / trade["adjClose"]
                            trade["score"] = score
                            trade["d"] = d
                            trade["seat"] = seat
                            trades.append(trade)
                            date = sell_trade["date"] + timedelta(days=1)
                        except Exception as e:
                            continue
                    date = date + timedelta(days=1)
            except Exception as e:
                date = date + timedelta(days=1)

  0%|                                                                                                          | 0/3 [00:00<?, ?it/s]<ipython-input-8-50f363775b30>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade["sell_date"] = sell_trade["date"]
<ipython-input-8-50f363775b30>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade["sell_price"] = sell_trade["adjClose"]
<ipython-input-8-50f363775b30>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [9]:
final_sim[(final_sim["ticker"]==ticker) &  (final_sim["date"] == beginning_date)]

NameError: name 'beginning_date' is not defined

In [10]:
t = pd.DataFrame(trades)

In [11]:
a = []
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in tqdm(range(70,100,10)):
        score_trades = t[(t["score"]==score) & (t["d"]==d)]
        cash = []
        for seat in range(seats):
            initial = float(100 / seats)
            seat_trades = score_trades[score_trades["seat"]==seat]
            for delta in seat_trades["delta"]:
                initial = initial * (1+delta)
                cash.append(initial)
            analysis = {}
            analysis["score"] = score
            analysis["trades"] = seat_trades.index.size
            analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
            seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
            seat_trades["hpr"] =[x.days for x in seat_trades["hpr"]]
            analysis["hpr"] = seat_trades["hpr"].mean()
            analysis["seat"] = seat
            analysis["d"] = d
            analysis["pv"] = initial
            a.append(analysis)

  0%|                                                                                                          | 0/3 [00:00<?, ?it/s]<ipython-input-11-3c022a3c3efc>:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
<ipython-input-11-3c022a3c3efc>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
<ipython-input-11-3c022a3c3efc>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [12]:
final = pd.DataFrame(a)

In [13]:
final

,score,trades,winrate,hpr,seat,d,pv
0,70,198,0.772727,3.747475,0,0.00,26.523874
1,70,197,0.705584,9.177665,1,0.00,93.736402
2,70,197,0.715736,14.192893,2,0.00,140.389334
3,80,200,0.780000,3.680000,0,0.00,53.381699
4,80,199,0.708543,8.929648,1,0.00,159.102246
5,80,198,0.717172,13.969697,2,0.00,105.418409
6,90,204,0.759804,3.901961,0,0.00,102.039281
7,90,195,0.789744,8.456410,1,0.00,86.998481
8,90,188,0.750000,13.281915,2,0.00,944.235711
9,70,108,0.611111,8.620370,0,0.05,207.803336


In [14]:
hl = final.pivot_table(index=["score","d"],columns="seat",values="pv")

In [15]:
report = final.merge(hl.reset_index(),on=["score","d"]).drop(["pv","seat"],axis=1).groupby(["score","d"]).mean()
report["pv"] = [sum([row[1][x] for x in range(seats)]) for row in report.iterrows()]

In [16]:
report.sort_values("pv",ascending=False).reset_index()

,score,d,trades,winrate,hpr,0,1,2,pv
0,90,0.00,195.666667,0.766516,8.546762,102.039281,86.998481,944.235711,1133.273473
1,70,0.05,107.333333,0.611803,18.381245,207.803336,190.024969,198.404506,596.232812
2,70,0.10,107.333333,0.611803,18.381245,207.803336,190.024969,198.404506,596.232812
3,90,0.05,101.666667,0.602334,18.274357,235.556042,193.431802,124.534680,553.522523
4,90,0.10,101.666667,0.602334,18.274357,235.556042,193.431802,124.534680,553.522523
5,80,0.00,199.000000,0.735238,8.859782,53.381699,159.102246,105.418409,317.902354
6,80,0.05,107.333333,0.565164,18.315106,143.465379,38.330779,116.081780,297.877938
7,80,0.10,107.333333,0.565164,18.315106,143.465379,38.330779,116.081780,297.877938
8,70,0.00,197.333333,0.731349,9.039344,26.523874,93.736402,140.389334,260.649609


In [17]:
benchmark = t[(t["score"]==90) & (t["d"]==0)]

In [18]:
timelines =[]
for seat in tqdm(range(seats)):
    seat_trades = benchmark[benchmark["seat"]==seat]
    initial = float(100/seats)
    timeline = []
    for row in seat_trades.iterrows():
        initial = initial * (1+row[1]["delta"])
        timeline.append(initial)
    seat_trades["pv"] = timeline
    timelines.append(seat_trades)

  0%|                                                                                                          | 0/3 [00:00<?, ?it/s]<ipython-input-18-81cbf75264c0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["pv"] = timeline
100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 74.98it/s]


In [19]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()
sp5.rename(columns={"Symbol":"ticker"},inplace=True)
benchmark = benchmark.merge(sp5,on="ticker",how="left")

In [20]:
sp5.columns

Index(['_id', 'ticker', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK',
       'Founded'],
      dtype='object')

In [21]:
# industry check
benchmark.groupby(["year","quarter","GICS Sector"]).mean().sort_values("delta",ascending=False)

adjClose  \
year quarter GICS Sector                          
2017 1       Information Technology   16.480586   
2018 2       Real Estate              11.066021   
2020 4       Financials               10.623694   
2015 1       Real Estate             117.740182   
2020 4       Real Estate              12.763012   
     2       Consumer Discretionary  135.151547   
2013 2       Communication Services  101.170000   
             Information Technology    6.448072   
2015 1       Consumer Staples         34.979329   
2019 1       Consumer Discretionary  114.574955   
2018 2       Materials                97.320318   
2013 4       Communication Services   17.713436   
2015 1       Materials               106.841080   
2012 4       Health Care               9.578333   
2014 4       Information Technology  120.267917   
2020 3       Communication Services   13.780508   
2015 3       Consumer Staples        112.994367   
2019 2       Health Care              38.977926   
2016 1       Communication Services  102.679455   
2017 4       Consumer Discretionary   12.912343   
2012 2       Consumer Staples         11.459307   
2014 2       Materials               103.007171   
2016 3       Health Care              24.286481   
             Information Technology   46.513373   
2013 4       Energy                  175.152464   
2019 3       Utilities               129.528889   
2014 2       Consumer Discretionary  135.328340   
2013 4       Information Technology  127.894169   
2016 2       Communication Services  113.845410   
2013 4       Materials                31.942159   
2017 2       Industrials              27.786748   
2014 1       Materials                24.987219   
2016 4       Real Estate              18.681778   
2018 1       Industrials             101.968318   
2017 3       Industrials              18.115530   
     1       Real Estate              15.668161   
2019 4       Financials               14.790125   
2018 3       Real Estate              18.868563   
     4       Health Care             109.706376   
2013 3       Information Technology    7.099967   
2015 3       Information Technology  121.673411   
2018 2       Health Care              30.237962   
2017 1       Financials               11.470671   
2012 3       Consumer Discretionary    6.151313   
2015 2       Consumer Staples         29.328750   
     4       Communication Services   46.816802   
2019 3       Materials               211.239176   
2018 1       Communication Services  107.655144   
2014 2       Communication Services  138.006000   
2012 2       Consumer Discretionary   74.487693   
2015 2       Information Technology  156.460000   
2014 3       Industrials              11.495554   
2017 1       Materials                17.844457   
2020 1       Information Technology  106.451889   
2019 3       Financials               79.546307   
2017 2       Information Technology   15.360167   

                                     weekly_price_regression_prediction  \
year quarter GICS Sector                                                  
2017 1       Information Technology                           20.096453   
2018 2       Real Estate                                      13.663418   
2020 4       Financials                                       13.428919   
2015 1       Real Estate                                     120.741005   
2020 4       Real Estate                                      18.974891   
     2       Consumer Discretionary                          138.628458   
2013 2       Communication Services                          124.603723   
             Information Technology                            8.496621   
2015 1       Consumer Staples                                 36.466111   
2019 1       Consumer Discretionary                          120.478108   
2018 2       Materials                                       123.186813   
2013 4       Communication Services                           19.199016   
2015 1       Materials        

In [22]:
benchmark.groupby(["year","quarter","ticker"]).mean().sort_values("delta",ascending=False).reset_index().merge(sp5[["ticker","Security"]],on="ticker",how="left").head(20)

,year,quarter,ticker,adjClose,weekly_price_regression_prediction,score,delta,sell_price,d,seat,CIK,Security
0,2017,1,ENPH,1.325000,1.634911,90.0,0.129464,1.430000,0.0,1.000000,1463101.0,Enphase Energy
1,2019,2,CTLT,40.070000,58.337936,90.0,0.108310,44.410000,0.0,2.000000,1596783.0,Catalent
2,2015,1,KR,31.087750,32.612799,90.0,0.106246,34.390684,0.0,1.000000,56873.0,Kroger
3,2018,2,KIM,11.066021,13.663418,90.0,0.092900,12.094058,0.0,2.000000,879101.0,Kimco Realty
4,2013,2,MU,9.290000,11.553764,90.0,0.089832,10.122500,0.0,2.000000,723125.0,Micron Technology
5,2017,4,ETSY,15.760000,93.202225,90.0,0.088198,17.150000,0.0,2.000000,1370637.0,Etsy
6,2020,2,TSLA,140.264000,141.719604,90.0,0.085368,152.238000,0.0,0.000000,1318605.0,Tesla
7,2013,3,MU,12.690000,18.338770,90.0,0.081954,13.730000,0.0,2.000000,723125.0,Micron Technology
8,2013,2,HPQ,8.471492,10.494182,90.0,0.075131,9.118783,0.0,2.000000,47217.0,HP
9,2016,3,SWKS,56.420355,61.932584,90.0,0.073247,60.552968,0.0,2.000000,4127.0,Skyworks Solutions
